### Run BASiCS for estimation method comparisons

In [1]:
library(Seurat)
library(SeuratData)
library(cowplot)
library(patchwork)
library(scHOT)
library(tidyverse)
library(dplyr)
library(BiocParallel)
library(readr)
library(Matrix)

Attaching SeuratObject

Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element 

In [2]:
library(Seurat)
library(SeuratData)
library(SeuratDisk)

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat



In [3]:
library(BASiCS)

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQR

### Convert h5ad to seurat objects for simulations

In [4]:
setwd('/data_volume/memento/simulation/inference/')


In [5]:

filenames <- c('de','dv')

for (f in filenames){
    filename = paste(
        f,
        '_filtered.h5ad',
        sep=''
    )
    Convert(filename, dest = "h5seurat", overwrite = TRUE, verbose = TRUE)
        
}

Warning message:
“Unknown file type: h5ad”
Warning message:
“'assay' not set, setting to 'RNA'”
Creating h5Seurat file for version 3.1.5.9900

Adding X as data

Adding X as counts

Adding meta.features from var

Adding memento to miscellaneous data

Warning message:
“Unknown file type: h5ad”
Warning message:
“'assay' not set, setting to 'RNA'”
Creating h5Seurat file for version 3.1.5.9900

Adding X as data

Adding X as counts

Adding meta.features from var

Adding memento to miscellaneous data



### BASiCS for simulations

In [6]:
runBASiCS <- function(seurat, group){
    Idents(seurat) <- 'ct_real'
    dat <- subset(x = seurat, idents = group)
    dat$batch <- sample(1:2, dim(dat)[2], replace=TRUE)
    dat.sce <- SingleCellExperiment(
        assays=list(counts = GetAssayData(dat)),
        colData = data.frame(BatchInfo = dat$batch))
    Chain <- BASiCS_MCMC(
      dat.sce,
      N = 20000, Thin = 20, Burn = 10000, WithSpikes = FALSE, SubsetBy = 'cell',
      PrintProgress = TRUE, Regression = TRUE,Threads = getOption("Ncpus", 42),)
    return(Chain)
}

In [ ]:
f <- 'dv'
print(f)

infilename = paste(
    f,
    '_filtered.h5seurat',
    sep=''
)

data_obj <- LoadH5Seurat(infilename, misc=F, verbose = FALSE)
print(dim(data_obj))

A_chain <- runBASiCS(data_obj, 'A')
saveRDS(A_chain, file = paste(f, "_A_filtered_chain.rds", sep=""))
B_chain <- runBASiCS(data_obj, 'B')
saveRDS(B_chain, file = paste(f, "_B_filtered_chain.rds", sep=""))


[1] "dv"


Validating h5Seurat file



[1] 1625 2000


Running no spikes BASiCS sampler (regression case) ... 




In [45]:
A_chain <- readRDS(paste('dv', "_A_filtered_chain.rds", sep=""))
B_chain <- readRDS(paste('dv', "_B_filtered_chain.rds", sep=""))

Test <- BASiCS_TestDE(
  Chain1 = A_chain, Chain2 = B_chain,
  GroupLabel1 = "A", GroupLabel2 = "B",
  EpsilonM = 0, EpsilonD =0,
  EpsilonR = 0,
  EFDR_M = 1, EFDR_D = 1,EFDR_R=NULL, k=1,ProbThresholdR=0,
  Offset = TRUE, PlotOffset = TRUE, Plot = TRUE
)

dv <- as.data.frame(Test, Parameter = "ResDisp")

-------------------------------------------------------------
Log-fold change thresholds are now set in a log2 scale. 
Original BASiCS release used a natural logarithm scale.

-------------------------------------------------------------
Offset estimate: 0.9902
(ratio A vs B).
-------------------------------------------------------------


For Differential mean task:
the posterior probability threshold chosen via EFDR calibrationis too low. Probability threshold automatically set equal to'ProbThresholdM'.

For Differential dispersion task:
the posterior probability threshold chosen via EFDR calibrationis too low. Probability threshold automatically set equal to'ProbThresholdD'.

EFDR = NULL forDifferential residual dispersion task:
Probability threshold automatically set equal to'ProbThresholdR'.



In [47]:
write_csv(dv, file = 'dv_basics.csv')


### run BASiCS for smFISH comparison

In [3]:
setwd('/data_volume/memento/saver/')
data_path <- '/data_volume/memento/saver/'

In [10]:
Convert('dropseq.h5ad', dest = "h5seurat", overwrite = TRUE, verbose = FALSE)

Warning message:
“Unknown file type: h5ad”
Warning message:
“'assay' not set, setting to 'RNA'”
Creating h5Seurat file for version 3.1.5.9900



In [12]:
runBASiCS_dropseq <- function(seurat, stat){
    Idents(seurat) <- 'group'
    dat <- seurat
    dat$batch <- 1
    dat.sce <- SingleCellExperiment(
    assays=list(counts = GetAssayData(dat)),
    colData = data.frame(BatchInfo = dat$batch))
    Chain <- BASiCS_MCMC(
      dat.sce,
      N = 1000, Thin = 10, Burn = 500, WithSpikes = FALSE, SubsetBy = 'cell',
      PrintProgress = FALSE, Regression = TRUE,Threads = getOption("Ncpus", 10),)
    df = as.data.frame(displayChainBASiCS(Chain, Param = stat))
    return(df)
}

In [10]:
data_obj <- LoadH5Seurat('dropseq.h5seurat', verbose = FALSE)

mu <- runBASiCS_dropseq(data_obj, 'mu')
write_csv(mu, file = 'dropseq_BASiCS_mu.csv')

delta <- runBASiCS_dropseq(data_obj, 'delta')
write_csv(delta, file = 'dropseq_BASiCS_delta.csv')

ERROR: Error: Cannot find file dropseq.h5seurat
